In [ ]:
from tqdm import tqdm
import pandas as pd
from transformers import pipeline


df_true = pd.read_csv("../data/topics_evaluation.csv")

df = df_true[df_true['class'] != 'unknown']
print(df)

In [ ]:

classifier = pipeline("zero-shot-classification", 
                      model="MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli", 
                      device=0,
                      batch_size=8)

candidate_labels = [
    "women rights",
    "climate",
    "labor rights",
    "health care",
    "farmers",
    "environment",
    "public services",
    "palestine-israel conflict",
    "immigration",
    "unjust law enforcement",
    "ukraine-russia war",
    "discrimination",
    "education",
    "housing",
    "culture",
    "policies",
    "animal welfare",
    "pandemic",
    "lgbtq"
]

texts = df['notes'].tolist()

batch_size = 8
top_k = 3
predictions = []

for i in tqdm(range(0, len(texts), batch_size)):
    batch = texts[i:i + batch_size]
    results = classifier(batch, candidate_labels)

    if isinstance(results, dict):
        top_labels = results['labels'][:top_k]
        predictions.append(top_labels)
    else:
        for res in results:
            top_labels = res['labels'][:top_k]
            predictions.append(top_labels)

df['predicted_topics_top3'] = predictions

df.to_csv('../data/topics_zsc_top3.csv', index=False)

Device set to use cuda:0
  0%|          | 0/86 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [6]:
import plotly.graph_objects as go
from plotly.offline import iplot, init_notebook_mode
import plotly.express as px
import plotly.io as pio
init_notebook_mode(True)

fig = px.bar(x=df["predicted_topic"].value_counts().index,y=df["predicted_topic"].value_counts(),color=df["predicted_topic"].value_counts().index,text=df["predicted_topic"].value_counts())
fig.update_traces(hovertemplate="Category:'%{x}' Counted: %{y}")
fig.update_layout(title={"text":"Category Counts","x":0.5,"font":{"size":35}},xaxis={"title":"Category","showgrid":False},yaxis={"title":"Value","showgrid":False},plot_bgcolor="white",width=800,height=500,showlegend=False)
iplot(fig)

KeyError: 'predicted_topic'

In [1]:
from sklearn.metrics import classification_report, accuracy_score



y_true = df['class']

y_pred_top3 = df['predicted_topics_top3']

top3_correct = [true_label in pred_list for true_label, pred_list in zip(y_true, y_pred_top3)]
top3_accuracy = sum(top3_correct) / len(top3_correct)

print("Top-3 Accuracy:", top3_accuracy)

NameError: name 'df' is not defined

In [4]:
print(df_eval)

                                                 notes               class  \
0    On 17 November 2022, doctors and nurses affili...         health care   
1    On 13 January 2022, at the call of eight teach...           education   
2    On 24 August 2022, Ukrainian activists gathere...  ukraine-russia war   
3    On 17 November 2020, students gathered outside...            pandemic   
4    On 2 May 2020, members of the newly-born movem...         health care   
..                                                 ...                 ...   
678  On 24 June 2022, an unknown number of farmers ...             farmers   
679  On 27 November 2024, Diageo workers staged a p...        labor rights   
680  On 20 June 2023, cleaning, catering, security ...        labor rights   
681  On 17 October 2024, around 30 undocumented mig...             housing   
682  On 25 May 2020, law enforcers in Slupia stoppe...            pandemic   

    real class         predicted_topic  
0      unknown        